In [1]:
import scispacy
import spacy
from spacy import displacy
import en_ner_bionlp13cg_md
import pandas as pd

df = pd.read_csv("patentAbstracts.csv")
pnumList = df['pnum'].tolist()
abstractList = df['abstract'].tolist()

nlp_bi = en_ner_bionlp13cg_md.load()

def getGeneNames(pnumList, abstractList):
    i = 0
    table= {"pnum":[], "Entity":[]}
    for doc in nlp_bi.pipe(abstractList):
        pnum = pnumList[i]
        for x in doc.ents:
            if x.label_=='GENE_OR_GENE_PRODUCT':
                table["pnum"].append(pnum)
                table["Entity"].append(x.text)
        i +=1
    return table

table = getGeneNames(pnumList, abstractList)

trans_df = pd.DataFrame(table)
trans_df.to_csv ("Entity_all.csv", index=False)


In [1]:
import scispacy
from scispacy.umls_linking import UmlsEntityLinker
import spacy
from spacy import displacy
import en_ner_bionlp13cg_md
import pandas as pd

df = pd.read_csv("patentAbstracts.csv")
pnumList = df['pnum'].tolist()
abstractList = df['abstract'].tolist()

nlp_bi = en_ner_bionlp13cg_md.load()
linker = UmlsEntityLinker(resolve_abbreviations=True, max_entities_per_mention=1)
nlp_bi.add_pipe(linker)

def getGeneNames(pnumList, abstractList):
    i = 0
    table= {"pnum":[], "Entity":[], "Type":[], "FullName":[]}
    for doc in nlp_bi.pipe(abstractList):
        pnum = pnumList[i]
        for ent in doc.ents:
            if ent.label_=='GENE_OR_GENE_PRODUCT':
                for umls_ent in ent._.umls_ents:
                    table["pnum"].append(pnum)
                    table["Entity"].append(ent.text)
                    info = str(linker.umls.cui_to_entity[umls_ent[0]])
                    lines = info.split('\n')
                    cuiandname = lines[0].split(', ')
                    fname = cuiandname[1][6:]
                    tui = lines[2][8:]
                    table["Type"].append(tui)
                    table["FullName"].append(fname)
        i +=1
    return table

table = getGeneNames(pnumList, abstractList)

trans_df = pd.DataFrame(table)
trans_df.to_csv ("Entity_umls.csv", index=False)